In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

from scispacy.data_util import read_full_med_mentions, read_ner_from_tsv
from scispacy.per_class_scorer import PerClassScorer
from scispacy.train_utils import evaluate_ner

import en_core_sci_sm


In [2]:
def train_ner(output_dir: str,
              train_data_path: str,
              dev_data_path: str,
              test_data_path: str,
              run_test: bool = None,
              model: str = None,
              n_iter: int = 10,
              meta_overrides: str = None):

    util.fix_random_seed(util.env_opt("seed", 0))
    train_data = read_ner_from_tsv(train_data_path)
    dev_data = read_ner_from_tsv(dev_data_path)
    test_data = read_ner_from_tsv(test_data_path)
    os.makedirs(output_dir, exist_ok=True)
    if run_test:
        nlp = spacy.load(model)
        print("Loaded model '%s'" % model)
        evaluate_ner(nlp, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
        evaluate_ner(nlp, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))
    else:
        train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides)

In [3]:
def train(model, train_data, dev_data, test_data, output_dir, n_iter, meta_overrides):
    """Load the model, set up the pipeline and train the entity recognizer."""
    
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
#         nlp = spacy.blank('en')  # create blank Language class
        nlp = en_core_sci_sm.load()
        print("Created blank 'en_core_sci_sm' model")

    if meta_overrides is not None:
        metadata = json.load(open(meta_overrides))
        nlp.meta.update(metadata)

    original_tokenizer = nlp.tokenizer

#     nlp.tokenizer = nlp_en.tokenizer

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names and "parser" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="parser")
    elif 'ner' not in nlp.pipe_names and "tagger" in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, after="tagger")
    elif 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in train_data:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    dropout_rates = util.decaying(util.env_opt('dropout_from', 0.2),
                                  util.env_opt('dropout_to', 0.2),
                                  util.env_opt('dropout_decay', 0.005))
    batch_sizes = util.compounding(util.env_opt('batch_from', 1),
                                   util.env_opt('batch_to', 32),
                                   util.env_opt('batch_compound', 1.001))

    optimizer = nlp.begin_training()
    best_epoch = 0
    best_f1 = 0
    for i in range(n_iter):
        print(str(i)+'--'+str(n_iter))
        random.shuffle(train_data)
        count = 0
        losses = {}
        total = len(train_data)

        with nlp.disable_pipes(*other_pipes):  # only train NER
            with tqdm.tqdm(total=total, leave=True) as pbar:
                for batch in minibatch(train_data, size=batch_sizes):
                    docs, golds = zip(*batch)
                    nlp.update(docs, golds, sgd=optimizer,
                               losses=losses, drop=next(dropout_rates))
                    pbar.update(len(batch))
                    if count % 100 == 0 and count > 0:
                        print('sum loss: %s' % losses['ner'])
                    count += 1

        # save model to output directory
        output_dir_path = Path(output_dir + "/" + str(i))
        if not output_dir_path.exists():
            output_dir_path.mkdir()

        with nlp.use_params(optimizer.averages):
            nlp.tokenizer = original_tokenizer
            nlp.to_disk(output_dir_path)
            print("Saved model to", output_dir_path)

        # test the saved model
        print("Loading from", output_dir_path)
        nlp2 = util.load_model_from_path(output_dir_path)
#         nlp2.tokenizer = nlp_en.tokenizer

        metrics = evaluate_ner(nlp2, dev_data)
        if metrics["f1-measure-overall"] > best_f1:
            best_f1 = metrics["f1-measure-overall"]
            best_epoch = i
    # save model to output directory
    best_model_path = Path(output_dir + "/" + "best")
    print(f"Best Epoch: {best_epoch} of {n_iter}")
    if os.path.exists(best_model_path):
        shutil.rmtree(best_model_path)
    shutil.copytree(os.path.join(output_dir, str(best_epoch)),
                    best_model_path)

    # test the saved model
    print("Loading from", best_model_path)
    nlp2 = util.load_model_from_path(best_model_path)
#     nlp2.tokenizer = nlp_en.tokenizer

    evaluate_ner(nlp2, dev_data, dump_path=os.path.join(output_dir, "dev_metrics.json"))
    evaluate_ner(nlp2, test_data, dump_path=os.path.join(output_dir, "test_metrics.json"))

In [4]:
datapath = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/relation_dataset/'
model_output_dir = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/'
train_data_path = datapath+'train.csv'
dev_data_path = datapath+'devel.csv'
test_data_path = datapath+'test.csv'
run_test = False
model_path = None
iterations = 10
meta_overrides = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/scispacy/data/EPMC_ner.json'

In [5]:
train_ner(model_output_dir,
              train_data_path,
              dev_data_path,
              test_data_path,
              run_test,
              model_path,
              iterations,
              meta_overrides)

  0%|          | 2/488 [00:00<00:39, 12.38it/s]

Created blank 'en_core_sci_sm' model
0--10


 21%|██        | 103/488 [00:07<00:28, 13.50it/s]

sum loss: 1286.907295449678


 42%|████▏     | 203/488 [00:14<00:20, 14.06it/s]

...

Evaluating 61 rows


  8%|▊         | 5/61 [00:00<00:01, 47.24it/s]

precision-untyped: 		 0.7919075144508665
recall-untyped: 		 0.7248677248677244
f1-measure-untyped: 		 0.756906077348016
precision-overall: 		 0.7745664739884388
recall-overall: 		 0.7089947089947086
f1-measure-overall: 		 0.7403314917126568
Evaluating 61 rows


100%|██████████| 61/61 [00:01<00:00, 47.50it/s]


precision-untyped: 		 0.7387387387387384
recall-untyped: 		 0.7999999999999996
f1-measure-untyped: 		 0.7681498829039309
precision-overall: 		 0.6568047337278102
recall-overall: 		 0.7449664429530196
f1-measure-overall: 		 0.6981132075471196


In [7]:
best_model_path = model_output_dir+'best/'

print("Loading from", best_model_path)
nlp2 = util.load_model_from_path(best_model_path)
doc = nlp2(text)


Loading from /nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/best/


In [16]:
import pandas as pd

df = pd.read_csv('/nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Notebooks/SpaCy NER and TextCat/agreed_.csv', names=['expert','sentence', 'label'])
agreed_df = df[['sentence', 'label']] 


In [32]:
text = agreed_df.iloc[30].sentence
# text = 'These results indicate the dentate gyrus is mostly comprised of mature neurons (NeuN), along with a smaller population of precursor cells (nestin) and newly differentiated neurons (DCX), which corresponds to prior findings examining the relative number of each cell population in the dentate gyrus, indicating the proportion of cells labeled by the sensor approximately reflects physiological proportions [21, 22]. '
sentence = nlp2(text)
print(text)
for ent in sentence.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Urinary TGF-β1 excretion increased in patients with crescentic nephritis (CN) (n = 15 p < 0.01) and with various types of proliferative GN (vGN) (n = 12 p < 0.05) compared to healthy subjects (n = 10).
increased 25 34 POSITIVE_REGULATION


In [34]:
!python -m spacy package /nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/best/ /nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/model

✔ Loaded meta.json from file
/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/best/meta.json
✔ Successfully created package 'en_ner_europepmc_md-0.2.5'
/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/relation_ner/model/en_ner_europepmc_md-0.2.5
To build the package, run `python setup.py sdist` in this directory.
